## Load Embeddings Model

In [1]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

/home/ubuntu/.local/share/virtualenvs/llm-homework-03-TuzjjQaC/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [18]:
q_v = embedding_model.encode(user_question)
print(q_v[0])

0.07822264


## Prepare the Documents

In [12]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
print(documents[0])

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}


In [17]:
# Filter the docs on just the ML course
documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]
print(len(documents))

375


## Create the Embeddings

In [19]:
import numpy as np

In [22]:
from tqdm.auto import tqdm

In [24]:
embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)
print(X.shape)

100%|██████████| 375/375 [00:27<00:00, 13.57it/s]

(375, 768)


## Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the question vector and the matrix X.

Since the vectors returned from the embedding model are already normalized, we can use the dot product to compute cosine similarity. 

In [51]:
scores = X.dot(q_v)
scores.sort()
print(scores.max()) # The top score

0.6506573


## Vector Search

In [38]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings # matrix, each row is a vector representation of the docs

    # Searches for most similar documents to the v_query
    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query) # Computes similarity score
        idx = np.argsort(-scores)[:num_results] # Sort scores in desc order, get top indices (up to num results)
        return [self.documents[i] for i in idx] # Return top docs

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(q_v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Evaluation

We will be implementing hit rate to evaluate the results. To do this we will be using the 'Ground Truth' dataset, which is a dataset that was created by interating through each document and then using an LLM to generate 5 questions for each FAQ document. 

In [40]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [41]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [54]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [55]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [63]:
def numpy_cosine_search(q):
    question = q['question']
    q_v = embedding_model.encode(question)
    return search_engine.search(q_v, num_results=5)

In [64]:
# Evaluates the search results that are received for each question in the ground_truth dataset
evaluate(ground_truth, numpy_cosine_search)

100%|██████████| 1830/1830 [00:39<00:00, 46.12it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}